In [9]:
import pandas as pd

# Etape 1: Lire le fichier CSV
df = pd.read_csv('archive/test/result/database_hierarchical_topics.csv')

# Etape 2: Enlever la première colonne
df_modifie = df.drop(df.columns[0], axis=1)

# Etape 3: Sauvegarder le DataFrame modifié dans un nouveau fichier CSV
df_modifie.to_csv('archive/test/result/database_hierarchical_topics.csv', index=False)



In [16]:
dossier = 'archive/test/result'
df = pd.read_csv(f"{dossier}/database_hierarchical_topics.csv")

df.loc[df['Parent_ID'] == 422, 'Parent_Name'].iloc[0]


'said_mr_would_trump_one'

In [27]:
import os
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML



def create_network(path_folder):
    news_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    news_net.barnes_hut()
    news_data = pd.read_csv(f"{path_folder}/database_update.csv")
    hierarchical_topics = pd.read_csv(f"{path_folder}/database_hierarchical_topics.csv")
    

    sources = pd.concat([hierarchical_topics["Parent_ID"], hierarchical_topics["Parent_ID"]], ignore_index=True)
    sources_names = pd.concat([hierarchical_topics["Parent_Name"], hierarchical_topics["Parent_Name"]], ignore_index=True)
    targets = pd.concat([hierarchical_topics["Child_Left_ID"], hierarchical_topics["Child_Right_ID"]], ignore_index=True)
    targets_names = pd.concat([hierarchical_topics["Child_Left_Name"], hierarchical_topics["Child_Right_Name"]], ignore_index=True)
    weights = pd.concat([hierarchical_topics["Distance"], hierarchical_topics["Distance"]], ignore_index=True)

    edge_data = zip(sources, sources_names, targets, targets_names, weights)

    for e in edge_data:
                    src = e[0]
                    src_names = e[1]
                    dst = e[2]
                    dst_names = e[3]
                    w = e[4]

                    news_net.add_node(src, src_names, title=src)
                    news_net.add_node(dst, dst_names, title=dst)
                    news_net.add_edge(src, dst, value=w)

    neighbor_map = news_net.get_adj_list()

    """# add neighbor data to node hover data
    for node in news_net.nodes:
                    print(node)
                    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                    node["value"] = len(neighbor_map[node["id"]])"""
    
    for node in news_net.nodes:
    # Assurez-vous que node["title"] est une chaîne de caractères
        node_title_str = str(node["title"])
        neighbors_str = ", ".join(str(neighbor) for neighbor in neighbor_map[node["id"]])
    # Utilisez la version chaîne de caractères de node["title"] pour la concaténation
        node["title"] = node_title_str + "\n Neighbors : [" + neighbors_str + "]"
        node["value"] = len(neighbor_map[node["id"]])


    

    news_net.toggle_physics(True)
    news_net.save_graph("graphs.html")
    


if __name__ == '__main__':

    dossier = 'archive/test/result'
    create_network(dossier)

    

Warning: When  cdn_resources is 'local' jupyter notebook has issues displaying graphics on chrome/safari. Use cdn_resources='in_line' or cdn_resources='remote' if you have issues viewing graphics in a notebook.